In [ ]:
# default_exp data.question_answering

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.question_answering

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for question/answering tasks.

In [ ]:
#export
import ast
from functools import reduce

from blurr.utils import *
from blurr.data.core import *

import torch
from transformers import *
from fastai2.text.all import *

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Question/Answering tokenization, batch transform, and DataBlock methods

Question/Answering tasks are models that require two text inputs (a context that includes the answer and the question).  The objective is to predict the start/end tokens of the answer in the context)

In [ ]:
path = Path('./')
squad_df = pd.read_csv(path/'squad_sample.csv'); len(squad_df)

1000

We've provided a simple subset of a pre-processed SQUADv2 dataset below just for demonstration purposes. There is a lot that can be done to make this much better and more fully functional.  The idea here is just to show you how things can work for tasks beyond sequence classification. 

In [ ]:
squad_df.head(2)

,id,title,context,question,answers,ds_type,answer_text,is_impossible
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",train,in the late 1990s,False
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",train,singing and dancing,False


In [ ]:
task = HF_TASKS_AUTO.QuestionAnswering

pretrained_model_name = 'roberta-base' #'xlm-mlm-ende-1024'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name, task=task)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [ ]:
#export
def pre_process_squad(row, hf_arch, hf_tokenizer):
    context, qst, ans = row['context'], row['question'], row['answer_text']
    
    tok_kwargs = {}
    if (hasattr(hf_tokenizer, 'add_prefix_space')): tok_kwargs['add_prefix_space'] = True
    
    if(hf_tokenizer.padding_side == 'right'):
        tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(qst, context, **tok_kwargs))
    else:
        tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(context, qst, **tok_kwargs))
                                                                       
    tok_ans = hf_tokenizer.tokenize(str(row['answer_text']), **tok_kwargs)
    
    start_idx, end_idx = 0,0
    for idx, tok in enumerate(tok_input):
        try:
            if (tok == tok_ans[0] and tok_input[idx:idx + len(tok_ans)] == tok_ans): 
                start_idx, end_idx = idx, idx + len(tok_ans)
                break
        except: pass
            
    row['tokenized_input'] = tok_input
    row['tokenized_input_len'] = len(tok_input)
    row['tok_answer_start'] = start_idx
    row['tok_answer_end'] = end_idx
    
    return row

The `pre_process_squad` method is structured around how we've setup the squad DataFrame above.

In [ ]:
squad_df = squad_df.apply(partial(pre_process_squad, hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), axis=1)

In [ ]:
max_seq_len= 128

In [ ]:
squad_df = squad_df[(squad_df.tok_answer_end < max_seq_len) & (squad_df.is_impossible == False)]

In [ ]:
#hide
squad_df.head(2)

,id,title,context,question,answers,ds_type,answer_text,is_impossible,tokenized_input,tokenized_input_len,tok_answer_start,tok_answer_end
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",train,in the late 1990s,False,"[<s>, ĠWhen, Ġdid, ĠBeyon, ce, Ġstart, Ġbecoming, Ġpopular, ?, </s>, </s>, ĠBeyon, cÃ©, ĠG, is, elle, ĠKnow, les, -, Carter, Ġ(/, bi, Ë, Ĳ, ËĪ, j, É, Ĵ, n, se, É, ª, /, Ġbee, -, Y, ON, -, say, ), Ġ(, born, ĠSeptember, Ġ4, ,, Ġ1981, ), Ġis, Ġan, ĠAmerican, Ġsinger, ,, Ġsong, writer, ,, Ġrecord, Ġproducer, Ġand, Ġactress, ., ĠBorn, Ġand, Ġraised, Ġin, ĠHouston, ,, ĠTexas, ,, Ġshe, Ġperformed, Ġin, Ġvarious, Ġsinging, Ġand, Ġdancing, Ġcompetitions, Ġas, Ġa, Ġchild, ,, Ġand, Ġrose, Ġto, Ġfame, Ġin, Ġthe, Ġlate, Ġ1990, s, Ġas, Ġlead, Ġsinger, Ġof, ĠR, &, B, Ġgirl, -, group, ĠDestiny, ...]",185,84,89
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",train,singing and dancing,False,"[<s>, ĠWhat, Ġareas, Ġdid, ĠBeyon, ce, Ġcompete, Ġin, Ġwhen, Ġshe, Ġwas, Ġgrowing, Ġup, ?, </s>, </s>, ĠBeyon, cÃ©, ĠG, is, elle, ĠKnow, les, -, Carter, Ġ(/, bi, Ë, Ĳ, ËĪ, j, É, Ĵ, n, se, É, ª, /, Ġbee, -, Y, ON, -, say, ), Ġ(, born, ĠSeptember, Ġ4, ,, Ġ1981, ), Ġis, Ġan, ĠAmerican, Ġsinger, ,, Ġsong, writer, ,, Ġrecord, Ġproducer, Ġand, Ġactress, ., ĠBorn, Ġand, Ġraised, Ġin, ĠHouston, ,, ĠTexas, ,, Ġshe, Ġperformed, Ġin, Ġvarious, Ġsinging, Ġand, Ġdancing, Ġcompetitions, Ġas, Ġa, Ġchild, ,, Ġand, Ġrose, Ġto, Ġfame, Ġin, Ġthe, Ġlate, Ġ1990, s, Ġas, Ġlead, Ġsinger, Ġof, ĠR, &, ...]",190,77,80


In [ ]:
vocab = dict(enumerate(range(max_seq_len)));

In [ ]:
#export
class HF_QuestionAnswerInput(list): pass

We'll return a `HF_QuestionAnswerInput` from our `HF_BatchTransform` so that we can customize the show_batch/results methods for this task.

In [ ]:
#export
class HF_QABatchTransform(HF_BatchTransform):
    def __init__(self, hf_arch, hf_tokenizer, **kwargs):
        super().__init__(hf_arch, hf_tokenizer, HF_QuestionAnswerInput, **kwargs)
        
    def encodes(self, samples):  
        samples = super().encodes(samples)
        for s in samples:
            # cls_index: location of CLS token (used by xlnet and xlm); is a list.index(value) for pytorch tensor's
            s[0]['cls_index'] = (s[0]['input_ids'] == self.hf_tokenizer.cls_token_id).nonzero()[0]
            # p_mask: mask with 1 for token than cannot be in the answer, else 0 (used by xlnet and xlm)
            s[0]['p_mask'] = s[0]['special_tokens_mask']
            
        return samples

By overriding `HF_BatchTransform` we can add other inputs to each example for this particular task.

In [ ]:
hf_batch_tfm = HF_QABatchTransform(hf_arch, hf_tokenizer)

blocks = (
    HF_TextBlock(hf_arch, hf_tokenizer, max_length=max_seq_len, truncation='only_second', 
                 hf_batch_tfm=hf_batch_tfm, 
                 tok_kwargs={ 'return_special_tokens_mask': True }), 
    CategoryBlock(vocab=vocab),
    CategoryBlock(vocab=vocab)
)

dblock = DataBlock(blocks=blocks, 
                   get_x=lambda x: (x.question, x.context),
                   get_y=[ColReader('tok_answer_start'), ColReader('tok_answer_end')],
                   splitter=RandomSplitter(),
                   n_inp=1)

In [ ]:
dls = dblock.dataloaders(squad_df, bs=4)

In [ ]:
b = dls.one_batch(); len(b), len(b[0]), len(b[1]), len(b[2])

(3, 5, 4, 4)

In [ ]:
b[0]['input_ids'].shape, b[0]['attention_mask'].shape, b[1].shape, b[2].shape

(torch.Size([4, 128]), torch.Size([4, 128]), torch.Size([4]), torch.Size([4]))

In [ ]:
#export
@typedispatch
def show_batch(x:HF_QuestionAnswerInput, y, samples, hf_tokenizer=None, ctxs=None, max_n=6, **kwargs):  
    res = L()
    for sample, input_ids, start, end in zip(samples, x[0], *y):
        txt = sample[0]
        ans_toks = hf_tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=False)[start:end]
        res.append((txt, (start.item(),end.item()), hf_tokenizer.convert_tokens_to_string(ans_toks)))
                       
    display_df(pd.DataFrame(res, columns=['text', 'start/end', 'answer'])[:max_n])
    return ctxs

The `show_batch` method above allows us to create a more interpretable view of our question/answer data.

In [ ]:
dls.show_batch(hf_tokenizer=hf_tokenizer, max_n=2)

,text,start/end,answer
0,"Where did Beyonce perform in 2011? In 2011, documents obtained by WikiLeaks revealed that Beyoncé was one of many entertainers who performed for the family of Libyan ruler Muammar Gaddafi. Rolling Stone reported that the music industry was urging them to return the money they earned for the concerts; a spokesperson for Beyoncé later confirmed to The Huffington Post that she donated the money to the Clinton Bush Haiti Fund. Later that year she became the first solo female artist to headline the main Pyramid stage at the 2011 Glastonbury Festival in over twenty years, and was named the highest-paid performer in the world per minute.","(103, 107)",Glastonbury Festival
1,"What are Beyoncé's backup singers called? In 2006, Beyoncé introduced her all-female tour band Suga Mama (also the name of a song in B'Day) which includes bassists, drummers, guitarists, horn players, keyboardists and percussionists. Her background singers, The Mamas, consist of Montina Cooper-Donnell, Crystal Collins and Tiffany Moniqué Riddick. They made their debut appearance at the 2006 BET Awards and re-appeared in the music videos for ""Irreplaceable"" and ""Green Light"". The band have supported Beyoncé in most subsequent live performances","(63, 66)",The Mamas


## Cleanup

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01a_data-language-modeling.ipynb.
Converted 01c_data-question-answering.ipynb.
Converted 01d_data-token-classification.ipynb.
Converted 01e_data-text-generation.ipynb.
Converted 02_modeling-core.ipynb.
Converted 02_training-summarization.ipynb.
Converted 02a_modeling-language-modeling.ipynb.
Converted 02c_modeling-question-answering.ipynb.
Converted 02d_modeling-token-classification.ipynb.
Converted 02e_modeling-text-generation.ipynb.
Converted index.ipynb.
